## 4. Aufgaben für Neuronale Netze in Python 3.x

Um die folgenden Aufgaben erledigen zu können benötigen wir zunächst den passenden Quellcode für unser Objekt des neuronalen Netzes. 

Das Netz und diese Unterlagen basieren auf dem Buch: Rashid, Neuronale Netze selbst programmieren, O`Reilly-Verlag, 2018 

In [1]:
##################### 1. Import der benötigten Bibs ###################################

# 1.1 numpy math functions (arrays and more)
import numpy
# 1.2 scipy.special for the sigmoid function expit()
import scipy.special

##################### 2. Klasse des Neuronalen Netzes #################################

# 2. neural network class definition
class neuralNetwork:
    
    # 2.1 initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # 2.1.1 set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # 2.1.2 link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = (numpy.random.rand(self.hnodes, self.inodes) - 0.5)
        self.who = (numpy.random.rand(self.onodes, self.hnodes) - 0.5)
        #self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes,self.inodes))
        #self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes,self.hnodes))
        
        # 2.1.3 learning rate
        self.lr = learningrate
        
        # 2.1.4 activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass
    
    # 2.2 train the neural network
    def train(self, inputs_list, targets_list):
        # 2.2.1 convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # 2.2.2 calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # 2.2.3 calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # 2.2.4 calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # 2.2.5 calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # 2.2.6 output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # 2.2.7 hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # 2.2.8 update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # 2.2.9 update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass   
    
    # 2.3 query the neural network
    def query(self, inputs_list):
        # 2.3.1 convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # 2.3.2 calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # 2.3.3 calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # 2.3.4 calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # 2.3.5 calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # 2.3.6 Feedback of the neural net
        return final_outputs

In [2]:
##################### 3. Testobjekt der oberen Klasse #################################    
    
# 3.1 number of input, hidden and output nodes
input_nodes = 3
hidden_nodes = 3
output_nodes = 3

# 3.2 learning rate is 0.3
learning_rate = 0.3

# 3.3 create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [3]:
##################### 4. Testabfrage für einen Eingangswert von 1.0 0.5 und -1.5 ##############################
##################### Ziel ist es einen Ausgangswert zu bekommen ##############################################
n.query([1.0,0.5,-1.5])

array([[0.46110354],
       [0.44619811],
       [0.54505988]])

Das war der Stand aus Kapitel 2. Wir haben nun ein vollständiges neuronales Netz und wollen dies einsetzen.

Dafür benötigen wir noch ein paar weitere Zeilen Code.

In [4]:
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

In Zeile 4 wird zusätzlich noch mit dem Laden der Bibliothek matplotlib die Möglichkeit zur grafischen Ausgabe diverser Daten gegeben.

In [5]:
# load the training data CSV file into a list
training_data_file = open("data/my_training_dataset_table.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

# load the test data CSV file into a list
test_data_file = open("data/my_test_dataset_table.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

FileNotFoundError: [Errno 2] No such file or directory: 'data/my_training_dataset_table.csv'

In Zeile 5 laden wir die beiden Datensätze, welche sich auf unserer Festplatte befinden. Die Pfadangabe geht hierbei von dem Standort des Notebooks aus. Die Datensätze werden in eine Variable training_data_list bzw. test_data_list eingelesen.

In [6]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 5

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

NameError: name 'training_data_list' is not defined

In Zeile 6 teilen wir die Traingsdaten in ...

In [7]:
# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

NameError: name 'test_data_list' is not defined

In [8]:
# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  nan


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


## 3.1 Vollständiger Code in einem Block

In [9]:
#######################################################################################
##################### 1. Import der benötigten Bibs ###################################
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

#######################################################################################
##################### 2. Anpassung der Variablen ######################################
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.1

# epochs is the number of times the training data set is used for training
epochs = 5

# load the mnist training data CSV file into a list
training_data_file = open("data/mnist_dataset/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

# load the mnist test data CSV file into a list
test_data_file = open("data/mnist_dataset/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

#######################################################################################
##################### 3. Klasse des Neuronalen Netzes #################################
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

#######################################################################################
##################### 4. Erstellen eines Objekts der obigen Klasse ####################
    
# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

#######################################################################################
##################### 5. Das Netz basierend auf den Epochen trainieren ################
 

# train the neural network
for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

#######################################################################################
##################### 6. Das Netz auf Basis der Testdaten prüfen ######################

# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

#######################################################################################
##################### 7. Ausgabe der Genauigkeit des Netzes (Performance) #############

# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

#######################################################################################
#######################################################################################

performance =  0.9727
